In [208]:
import requests as re
import pandas as pd
import datetime
import altair as alt
import altair_latimes as lat

In [209]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Download
Source: [California Water Watch](https://cww.water.ca.gov/), PRISM

Some parameters

In [210]:
today = datetime.date.today()
today

datetime.date(2024, 2, 6)

In [211]:
one_year_ago = datetime.datetime.today() - datetime.timedelta(days=365)
one_year_ago

datetime.datetime(2023, 2, 6, 13, 38, 49, 548296)

In [212]:
atlasID="Statewide"
varKey="PPT_YTD"

Query

In [213]:
url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={today}&atlasID={atlasID}&varKey={varKey}'

In [214]:
data = re.get(url).json()['data']

Query one year ago

In [215]:
last_year_url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={one_year_ago}&atlasID={atlasID}&varKey={varKey}'

In [216]:
last_year_data = re.get(last_year_url).json()['data']

### Parse

In [217]:
def parsePrecipStats(data):
    columns = data['seriesNames']
    columns.insert(0, "date")
    
    df = pd.DataFrame(data['series'], columns=columns)
    df.columns = df.columns.str.lower()
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [218]:
df = parsePrecipStats(data)

In [219]:
last_year_df = parsePrecipStats(last_year_data)

## Get toplines

In [220]:
prcp_min=data['min']
prcp_min

2.48

In [221]:
prcp_avg=data['avg']
prcp_avg

12.92

In [222]:
prcp_max=data['max']
prcp_max

23.92

In [223]:
year_end_avg = data['yearEndAvg']
year_end_avg

23.74

### Plot

In [224]:
gauge = df[
    (~df.current.isna())
]

In [225]:
gauge = gauge[gauge.date == gauge.date.max()]

In [226]:
gauge["record"] = prcp_max

In [227]:
gauge["pct_of_avg"] = (gauge["current"] / gauge["average"]) * 100

Last year

In [228]:
total_last_year = last_year_df[ last_year_df.date == gauge.date.max() ].iloc[0]["current"]

In [229]:
gauge["one_year_ago"] = total_last_year

In [230]:
gauge

,date,current,average,record,pct_of_avg,one_year_ago
127,2024-02-05,12.6,12.84,23.92,98.130841,18.65


In [231]:
alt.Chart(
    pd.melt(gauge, id_vars="date", value_vars=["current","average","one_year_ago"])
).mark_bar().encode(
    x="value",
    y=alt.Y("variable", sort="-x"),
    color="variable"
).properties(
    title="Year-to-date precipitation in California compared to average",
    width=600,
    height=200
)

alt.Chart(...)

In [232]:
line_chart = pd.melt(df, id_vars="date", value_vars=["current","average"])

In [233]:
alt.Chart(line_chart).mark_line().encode(
    x="date:T",
    y="value",
    color="variable",
).properties(
    title=f"California precipitation in {today.year} compared to average",
    width=600
)

alt.Chart(...)

### Export

In [234]:
gauge.to_csv("../../data/processed/precipitation/latest.csv", index=False)